https://platform.openai.com/docs/guides/function-calling

In [1]:
from dotenv import load_dotenv
import os
import openai
from openai import OpenAI
import pandas as pd
import json

In [3]:
load_dotenv()

API_KEY = os.getenv('API_KEY')

In [4]:
client = OpenAI(
    api_key=os.environ.get("API_KEY"),
)

In [5]:
response = client.chat.completions.create(
    model = 'gpt-4-turbo', 
    messages = [{"role": "system", "content": 'You are a helpful assistant which informs about temperature'}, # system: tell the agent what role it needs to play
                {"role": "user", "content": 'Hey there'}],  
    temperature = 0.1
    )
print(response.choices[0].message.content)

Hello! How can I assist you today?


#### https://rapidapi.com/MeteosourceWeather/api/ai-weather-by-meteosource

In [211]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
import requests
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://yahoo-weather5.p.rapidapi.com/weather"

    querystring = {"location":location}

    headers = {
      "X-RapidAPI-Key": "b135cb7c3emsha69d64dddb4aae4p1c74b6jsn969c589c8bda",
      "X-RapidAPI-Host": "yahoo-weather5.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    response = json.dumps(response.json(), indent=4)
    
    print(response)
    return eval(response)['current_observation']

In [212]:
response=get_current_weather('Boston')

{
    "location": {
        "city": "Boston",
        "woeid": 2367105,
        "country": "United States",
        "lat": 42.358631,
        "long": -71.056702,
        "timezone_id": "America/New_York"
    },
    "current_observation": {
        "pubDate": 1717866629,
        "wind": {
            "chill": 79,
            "direction": "West",
            "speed": 13
        },
        "atmosphere": {
            "humidity": 45,
            "visibility": 21,
            "pressure": 1004.1
        },
        "astronomy": {
            "sunrise": "5:07 AM",
            "sunset": "8:20 PM"
        },
        "condition": {
            "temperature": 75,
            "text": "Fair"
        }
    },
    "forecasts": [
        {
            "day": "Sat",
            "date": 1717862400,
            "high": 75,
            "low": 63,
            "text": "Mostly Cloudy",
            "code": 28
        },
        {
            "day": "Sun",
            "date": 1717948800,
            "high": 71,

In [214]:
response

{'pubDate': 1717866629,
 'wind': {'chill': 79, 'direction': 'West', 'speed': 13},
 'atmosphere': {'humidity': 45, 'visibility': 21, 'pressure': 1004.1},
 'astronomy': {'sunrise': '5:07 AM', 'sunset': '8:20 PM'},
 'condition': {'temperature': 75, 'text': 'Fair'}}

In [216]:
# This is for the OPENAPI to understand
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

functions

[{'name': 'get_current_weather',
  'description': 'Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'type': 'string',
     'description': 'The city and state, e.g. San Francisco, CA'}},
   'required': ['location']}}]

In [217]:
user_message="Hi There"
messages=[]
messages.append({"role": "user", "content":user_message})
completion=client.chat.completions.create(
    model = "gpt-4-turbo",
    messages = messages
    
)

In [218]:
messages

[{'role': 'user', 'content': 'Hi There'}]

In [219]:
print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None)


In [220]:
user_message="What is the temperature of Boston now"

messages.append({"role": "user", "content": user_message})
completion=client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    functions=functions # add function
    
)

In [221]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Boston now'}]

We didn't attach the specific function, but OPENAI is able to understand what function to call and what parameters to use

In [222]:
response=completion.choices[0].message
response

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None)

In [223]:
import json
location=eval(response.function_call.arguments)['location'] # eval: convert string to JSON
print(location)

Boston, MA


In [224]:
function_name=response.function_call.name
function_name

'get_current_weather'

## Send the info on the function call and function response to GPT

In [225]:
messages.append(response)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

In [227]:
messages

[{'role': 'user', 'content': 'Hi There'},
 {'role': 'user', 'content': 'What is the temperature of Boston now'},
 ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{"location":"Boston, MA"}', name='get_current_weather'), tool_calls=None),
 {'role': 'function', 'name': 'get_current_weather', 'content': 'Boston, MA'}]

In [234]:
# extend conversation with function response
second_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=messages,
    functions=functions
)  # get a new response from GPT where it can see the function response

print(second_response.choices[0].message)

ChatCompletionMessage(content="I'll need a moment to fetch the latest weather data for Boston, MA. Please hang tight!\nIt seems there was an issue retrieving the temperature for Boston. Let's try that again.", role='assistant', function_call=None, tool_calls=None)
